In [ ]:
import ast
import os
os.chdir(os.pardir)

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from src import load_point_cloud
from src.metrics import _angle_error

from estimator import NormalEstimator

In [ ]:
data_path = 'data'

In [ ]:
with open(os.path.join(data_path, 'testset_no_noise.txt')) as f:
    testset_no_noise = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_low_noise.txt')) as f:
    testset_low_noise = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_med_noise.txt')) as f:
    testset_med_noise = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_high_noise.txt')) as f:
    testset_high_noise = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_vardensity_gradient.txt')) as f:
    testset_vardensity_gradient = [line.strip() for line in f]
with open(os.path.join(data_path, 'testset_vardensity_striped.txt')) as f:
    testset_vardensity_striped = [line.strip() for line in f]

In [ ]:
class_dispatcher = {'no_noise': testset_no_noise,
                    'low_noise': testset_low_noise,
                    'med_noise': testset_med_noise,
                    'high_noise': testset_high_noise,
                    'gradient': testset_vardensity_gradient,
                    'striped': testset_vardensity_striped}

In [ ]:
df_param_opt = pd.read_csv(os.path.join('estimator', 'param_opt.csv'),
                           index_col=0)
df_param_opt['params'] = df_param_opt['params'].apply(
    lambda row: ast.literal_eval(row)
)
df_param_opt

In [ ]:
def run_test(cls):
    res = {'class': [],
           'dataset': [],
           'params': [],
           'rmse': [],
           'pgp': []}
    for dataset in tqdm(class_dispatcher[cls]):
        xyz, n = load_point_cloud(dataset)
        params = df_param_opt.loc[
            df_param_opt['class'] == cls, 'params'
        ].item()
        estimator = NormalEstimator(**params).fit(xyz)
        n_estim = estimator.predict(xyz)
        e = _angle_error(n_estim, n, orient=False)
        rmse = np.sqrt(np.mean(e ** 2))
        pgp = []
        for alpha in alphas:
            pgp.append(np.mean(e < alpha))
        res['class'].append(cls)
        res['dataset'].append(dataset)
        res['params'].append(str(params))
        res['rmse'].append(rmse)
        res['pgp'].append(np.asarray(pgp))
    return res

In [ ]:
alphas = [0, 5, 10, 15, 20, 25, 30]

In [ ]:
res_no_noise = run_test('no_noise')

In [ ]:
res_med_noise = run_test('med_noise')

In [ ]:
res_low_noise = run_test('low_noise')

In [ ]:
res_high_noise = run_test('high_noise')

In [ ]:
res_gradient = run_test('gradient')

In [ ]:
res_striped = run_test('striped')

In [ ]:
df_no_noise = pd.DataFrame(res_no_noise)
df_low_noise = pd.DataFrame(res_low_noise)
df_med_noise = pd.DataFrame(res_med_noise)
df_high_noise = pd.DataFrame(res_high_noise)
df_gradient = pd.DataFrame(res_gradient)
df_striped = pd.DataFrame(res_striped)
df_test_res = pd.concat((df_no_noise,
                         df_low_noise,
                         df_med_noise,
                         df_high_noise,
                         df_gradient,
                         df_striped))
df_test_res

In [ ]:
save = False
fname = os.path.join('estimator', 'test_res.csv')

if save:
    df_test_res.to_csv(fname)

In [ ]:
group = df_test_res[['class', 'rmse', 'pgp']].groupby('class')
df_test_res_group = group.mean().sort_values('rmse')

In [ ]:
df_test_res_group